In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.nn.parameter import Parameter

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker


import numpy as np
import time
# from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import loadmat,savemat
from training_samples import trainingdata_uvw,trainingdata_T



#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda':
    print(torch.cuda.get_device_name())

cuda:1


In [2]:
R0 = 5 #mm
Rs = 19 #mm
# mu_vis = 0.3 
mu = 0.3 #Friction Coefficient (not viscosity)
# delta = 0.5
A = 6.41 #For slip factor
pi = np.pi
Omega = 300 #rpm
V = 2 #mm/s
F = 0.67 #mm
rho = 2700 * 1e-6 #g/mm3
k_B = 1.380649*1e-23 #J/K
R = 8.314 #J/(K.mol)
E_a = 205000 #J/mol #Q
alpha_sig = 52 #mm^2/N
# A = np.exp(27.78)
log_A = 27.78
n = 3.49
k = 0.167 #Thermal Conductivity #W/(mmK)
c_p = 0.897 #J/gK 
alpha_m = k/(rho*c_p)
T_a = 298.0


k_t = 0.0176 #W/(mmK)
c_p_t = 0.46 #J/gk
rho_t = 2700 * 1e-6 #g/mm3
alpha_t = k_t/(rho_t*c_p_t)

h_sides = 5*1e-6 #W/mm^2K
C_bot = 0.15*1e-6 #W/mm^2K^3

eeta = alpha_m/(alpha_m+alpha_t)

lb_xyz = np.array([-50.0,-20.0,-3.0])
ub_xyz = np.array([50.0,20.0,0.0])

lb_xyz_uvw = np.array([-20.0,-20.0,-3.0])
ub_xyz_uvw = np.array([20.0,20.0,0.0])


In [3]:
def train_step(xyz_coll,xyz_1, xyz_2, xyz_3, xyz_4,xyz_top,xyz_bot,f_hat,N_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xyz_coll,xyz_1, xyz_2, xyz_3, xyz_4,xyz_top,xyz_bot,f_hat,N_hat)
        loss.backward()

        return loss

    optimizer.step(closure)

In [4]:
def train_model(max_iter,rep,p_iters):
    print(rep)
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    xyz_coll,xyz_1, xyz_2, xyz_3, xyz_4,xyz_top,xyz_bot = trainingdata_uvw(N_B,N_f,lb_xyz,ub_xyz,rep*123)
    xyz_coll = torch.from_numpy(xyz_coll).float().to(device)
    xyz_1 = torch.from_numpy(xyz_1).float().to(device)
    xyz_2 = torch.from_numpy(xyz_2).float().to(device)
    xyz_3 = torch.from_numpy(xyz_3).float().to(device)
    xyz_4 = torch.from_numpy(xyz_4).float().to(device)
    
    xyz_top = torch.from_numpy(xyz_top).float().to(device)
    xyz_bot = torch.from_numpy(xyz_bot).float().to(device)
   
    f_hat = torch.zeros(xyz_coll.shape[0],1).to(device)
    N_hat = torch.zeros(xyz_top.shape[0],1).to(device)
    
    for i in range(max_iter):
#         if(i>0 and i%50==0):
#             _,_,_,_,_,xyz_top,xyz_bot = trainingdata(N_B,N_f,i*123)
#             # xyz_coll = torch.from_numpy(xyz_coll).float().to(device)
#             # xyz_1 = torch.from_numpy(xyz_1).float().to(device)
#             # xyz_2 = torch.from_numpy(xyz_2).float().to(device)
#             # xyz_3 = torch.from_numpy(xyz_3).float().to(device)
#             # xyz_4 = torch.from_numpy(xyz_4).float().to(device)

#             xyz_top = torch.from_numpy(xyz_top).float().to(device)
#             xyz_bot = torch.from_numpy(xyz_bot).float().to(device)

        train_step(xyz_coll,xyz_1, xyz_2, xyz_3, xyz_4,xyz_top,xyz_bot,f_hat,N_hat)

        loss_np = PINN.loss(xyz_coll,xyz_1, xyz_2, xyz_3, xyz_4,xyz_top,xyz_bot,f_hat,N_hat).cpu().detach().numpy()

        # print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
        print(i,"Train Loss",loss_np)

        # if(i>0 and i%25 ==0):
        #   pretrain(xyt_DBC,p_iters)


    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [5]:
from Seq_model_vPINN import Sequentialmodel
label = 'meltpool'
max_reps = 1
max_iter = 100
p_iters = 10

N_B = 6000
N_f = 5000

train_loss_full = []
test_mse_full = []
test_re_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

for reps in range(max_reps):

    train_loss = []
    test_mse_loss = []
    test_re_loss = []

    'Generate Training data'
    torch.manual_seed(reps*3)
     #Total number of collocation points


    # layers1 = np.array([3,50,50,50,4]) #9 hidden layers
    # layers2 = np.array([3,50,50,50,1]) #9 hidden layers
    # layers = np.array([3,60,60,60,60,60,5])
    layers = np.array([3,64,64,64,5])
    PINN = Sequentialmodel(layers,lb_xyz,ub_xyz)
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.25,
                              max_iter = 30,
                              max_eval = 50,
                              tolerance_grad = 1e-5,
                              tolerance_change = 1e-5,
                              history_size = 100,
                              line_search_fn = 'strong_wolfe')


    start_time = time.time()
    train_model(max_iter,reps,p_iters)

    label = "AFSD_vPINN"
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)


    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "label": label, "Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

cuda:1


Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=5, bias=True)
  )
)
0
0 Train Loss 3532.1504
1 Train Loss 760.0564
2 Train Loss 466.381
3 Train Loss 287.98132
4 Train Loss 194.53987
5 Train Loss 155.4568


KeyboardInterrupt: 

In [ ]:
A = torch.normal(mean=0, std=torch.arange(1, 0, -0.08)).reshape(-1,1)

In [ ]:
A.shape[0]

In [ ]:
[x_min,y_min,z_min] = lb_xyz
[x_max,y_max,z_max] = ub_xyz

x_min = -20.0
x_max = 20.0

x = np.linspace(x_min,x_max,100).reshape(-1,1)
y = np.linspace(y_min,y_max,100).reshape(-1,1)
#z = np.linspace(z_min,z_max,20).reshape(-1,1)
z = -0.125

X,Y,Z = np.meshgrid(x,y,z)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
Z = Z.flatten('F').reshape(-1,1)

xyz = np.hstack((X,Y,Z))
xyz_test_tensor = torch.from_numpy(xyz).float().to(device)

uvwp = PINN.forward1(xyz_test_tensor).cpu().detach().numpy()

In [ ]:
r_vel = np.sqrt(np.square(uvwp[:,0])+np.square(uvwp[:,1])+np.square(uvwp[:,2]))
# r_vel = np.sqrt(np.square(uvwp[:,2]))
# r_vel = uvwp[:,2]
fig,ax = plt.subplots()
im = ax.imshow((r_vel/1000).reshape(100,100),cmap = 'jet',extent = [-20,20,-20,20])
fig.colorbar(im)
ax.set_title('Resultant Velocity (Top) (m/s)')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
# plt.savefig('Res_u_top_Feb7.svg',format = 'svg',bbox_inches = 'tight')

In [ ]:
g = xyz_test_tensor.clone()
g.requires_grad = True

out_full = PINN.forward1(g) 
u = out_full[:,0:1]
v = out_full[:,1:2]
w = out_full[:,2:3]
p = out_full[:,3:4]


# print(T.shape)
T = PINN.forward2(g)

# p_xyz = autograd.grad(p,g,torch.ones([xyz_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
u_xyz = autograd.grad(u,g,torch.ones([xyz_test_tensor.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
v_xyz = autograd.grad(v,g,torch.ones([xyz_test_tensor.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
w_xyz = autograd.grad(w,g,torch.ones([xyz_test_tensor.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]

eps2_11 = torch.square(1/2*(2*u_xyz[:,0]))
eps2_12 = torch.square(1/2*(u_xyz[:,1] + v_xyz[:,0]))
eps2_13 = torch.square(1/2*(u_xyz[:,2] + w_xyz[:,0]))

eps2_21 = eps2_12
eps2_22 = torch.square(1/2*(2*v_xyz[:,1])) 
eps2_23 = torch.square(1/2*(v_xyz[:,2] + w_xyz[:,1]))

eps2_31 = eps2_13
eps2_32 = eps2_23 
eps2_33 = torch.square(1/2*(2*w_xyz[:,2]))

eps_e = torch.sqrt((2/3)*(eps2_11 + eps2_12 + eps2_13 + eps2_21 + eps2_22 + eps2_23 + eps2_31 + eps2_32 + eps2_33)).reshape(-1,1)


# Z = eps_e*torch.exp(E_a/(R*T))
# log_Z = torch.log(eps_e) + E_a/(R*T)
log_Z = torch.log(eps_e) + E_a/(R*T) #Simplification


W = (log_Z - log_A)/n



# sigma_e =  (1/alpha_sig)*torch.asinh(W) 
sigma_e = (1/alpha_sig)*(np.log(2)/n + W) #Approximation

#____________________________#
mu_vis = sigma_e/(3*eps_e)


eps_e = eps_e.cpu().detach().numpy()
sigma_e = sigma_e.cpu().detach().numpy()
mu_vis = mu_vis.cpu().detach().numpy()


In [ ]:
print(torch.__version__)

In [ ]:
fig,ax = plt.subplots()
im = ax.imshow(eps_e.reshape(100,100),cmap = 'jet',extent = [-20,20,-20,20])
fig.colorbar(im)
ax.set_title('Effective Stress (Pa)')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
# plt.savefig('Viscosity_Feb7.svg',format = 'svg',bbox_inches = 'tight')

In [ ]:
x = np.linspace(x_min,x_max,100).reshape(-1,1)
y =0
z = np.linspace(z_min,z_max,20).reshape(-1,1)
# z = z_max

X,Y,Z = np.meshgrid(x,y,z)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
Z = Z.flatten('F').reshape(-1,1)

xyz = np.hstack((X,Y,Z))
xyz_test_tensor = torch.from_numpy(xyz).float().to(device)

uvwp = PINN.forward1(xyz_test_tensor).cpu().detach().numpy()

In [ ]:
r_vel = np.sqrt(np.square(uvwp[:,0])+np.square(uvwp[:,1])+np.square(uvwp[:,2]))
fig,ax = plt.subplots()
im = ax.imshow(np.flip((r_vel/1000).reshape(20,100)),cmap = 'jet',extent = [-20,20,-3,0])
fig.colorbar(im,orientation = 'horizontal',shrink = 0.5)
ax.set_title('Resultant Velocity (CS) (m/s)')
ax.set_xlabel('$x$')
ax.set_ylabel('$z$')
# plt.savefig('Res_u_Front_Feb7.svg',format = 'svg',bbox_inches = 'tight')

In [ ]:
x = np.linspace(x_min,x_max,100).reshape(-1,1)
y = np.linspace(y_min,y_max,100).reshape(-1,1)
#z = np.linspace(z_min,z_max,20).reshape(-1,1)
z = -0.125

X,Y,Z = np.meshgrid(x,y,z)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
Z = Z.flatten('F').reshape(-1,1)

xyz = np.hstack((X,Y,Z))
xyz_test_tensor = torch.from_numpy(xyz).float().to(device)

T = PINN.forward2(xyz_test_tensor).cpu().detach().numpy()
fig,ax = plt.subplots()
im = ax.imshow(T.reshape(100,100),cmap = 'jet',extent = [-20,20,-20,20],vmax = 1000,vmin =300)
fig.colorbar(im)
ax.set_title('Temperature (top) (K)')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
# plt.savefig('Temp_top_Feb7.svg',format = 'svg',bbox_inches = 'tight')
# plt.scatter(X,Y,c=T,cmap = 'jet')

In [ ]:
np.min(T)

In [ ]:
x = np.linspace(x_min,x_max,100).reshape(-1,1)
y =0
z = np.linspace(z_min,z_max,20).reshape(-1,1)
# z = z_max

X,Y,Z = np.meshgrid(x,y,z)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
Z = Z.flatten('F').reshape(-1,1)

xyz = np.hstack((X,Y,Z))
xyz_test_tensor = torch.from_numpy(xyz).float().to(device)

uvwp = PINN.forward1(xyz_test_tensor).cpu().detach().numpy()

T = PINN.forward2(xyz_test_tensor).cpu().detach().numpy()
fig,ax = plt.subplots()
im = ax.imshow(np.flip(T.reshape(20,100)),cmap = 'jet',extent = [-20,20,-3,0])
fig.colorbar(im,orientation = 'horizontal',shrink = 0.5)
ax.set_title('Temperature (CS) (K)')
ax.set_xlabel('$x$')
ax.set_ylabel('$z$')
# plt.savefig('Temp_CS_Feb7.svg',format = 'svg',bbox_inches = 'tight')
# plt.scatter(X,Y,c=T,cmap = 'jet

In [ ]:
min(T)

In [ ]:
xyz_coll,xyz_1, xyz_2, xyz_3, xyz_4,xyz_top,xyz_bot = trainingdata(N_B,N_f,123)
    
r = np.sqrt(np.square(xyz_top[:,0]) + np.square(xyz_top[:,1]))
        
r_fr = (r<R0).reshape(-1,1)
r_ph = np.logical_and(r>=R0,r<=Rs).reshape(-1,1)
r_out = np.logical_not(r<=Rs).reshape(-1,1)

cos_theta = xyz_top[:,1]/r #
sin_theta = xyz_top[:,0]/r #

r = r.reshape(-1,1)
cos_theta = cos_theta.reshape(-1,1)
sin_theta = sin_theta.reshape(-1,1)

# out_top = self.forward1(g)

#BC1
u_true_ph = (1-delta)*(2*pi/60)*Omega*r*cos_theta
v_true_ph = (1-delta)*(2*pi/60)*Omega*r*sin_theta - V
w_true_ph = 0.0

#BC2
u_true_fr = (2*pi/60)*Omega*r*cos_theta
v_true_fr = (2*pi/60)*Omega*r*sin_theta - V
w_true_fr = -F

#OTHER
u_true_out = 0.0
v_true_out = -V
w_true_out = 0.0

u_true = u_true_fr*r_fr + u_true_ph*r_ph + u_true_out*r_out
v_true = v_true_fr*r_fr + v_true_ph*r_ph + v_true_out*r_out
w_true = w_true_fr*r_fr + w_true_ph*r_ph + w_true_out*r_out

# u = out_top[:,0:1]
# v = out_top[:,1:2]
# w = out_top[:,2:3]

In [ ]:
r_vel = np.sqrt(np.square(u_true)+np.square(v_true))
# plt.imshow(r_vel.reshape(100,100),cmap = 'jet')
plt.scatter(xyz_top[:,0],xyz_top[:,1],c=r_vel,cmap = 'jet')

In [ ]:
np.max(r_vel)